In [22]:
!pip install pandas boto3

In [ ]:
import boto3
import pandas as pd
from io import StringIO
import seaborn as sns
import geopandas as gpd
from google.colab import files
import matplotlib.pyplot as plt

In [24]:
s3_bucket = "bd-sars-cov2-data"
s3_client = boto3.client("s3")

In [26]:
# List all objects in the S3 bucket
response = s3_client.list_objects_v2(Bucket=s3_bucket)

# Check if there are any objects and list their keys (file paths)
if 'Contents' in response:
    for obj in response['Contents']:
        print(obj['Key'])
else:
    print("No objects found in the bucket.")

2020_sequences.acc
2020_sequences.csv
2020_sequences.fasta
First3m_2022_sequences.acc
First3m_2022_sequences.csv
First3m_2022_sequences.fasta
First6m_2021_sequences.acc
First6m_2021_sequences.csv
First6m_2021_sequences.fasta
Last6m_2021_sequences.acc
Last6m_2021_sequences.csv
Last6m_2021_sequences.fasta
Last6m_2022_sequences.acc
Last6m_2022_sequences.csv
Last6m_2022_sequences.fasta
Second3m_2022_sequences.acc
Second3m_2022_sequences.csv
Second3m_2022_sequences.fasta


In [27]:
# Read the metadata CSV files directly in S3 without downloading them into local sagemaker instance
# Look for CSV files in the S3 bucket -- This is our Metadata
response = s3_client.list_objects_v2(Bucket=s3_bucket, Prefix="")
# List all CSV files in the S3 bucket
csv_files = [obj['Key'] for obj in response.get('Contents', []) if obj['Key'].endswith('.csv')]

In [28]:
# Read the metadata CSV file directly from S3
metadata_file_key = csv_files[0]  # Pick the first CSV file in the list
csv_obj = s3_client.get_object(Bucket=s3_bucket, Key=metadata_file_key)
csv_content = csv_obj["Body"].read().decode("utf-8")

In [29]:
# Load CSV content into a pandas DataFrame
metadata_df = pd.read_csv(StringIO(csv_content))

# Rename the 'Accession' column for consistency
metadata_df.rename(columns={'Accession': 'accession_id'}, inplace=True)

# Inspect the metadata DataFrame
print(metadata_df.columns)
print(metadata_df.head())

/tmp/ipykernel_6938/1513178843.py:2: DtypeWarning: Columns (3,11,20,24) have mixed types. Specify dtype option on import or set low_memory=False.
  metadata_df = pd.read_csv(StringIO(csv_content))


Index(['accession_id', 'Organism_Name', 'GenBank_RefSeq', 'Assembly',
       'SRA_Accession', 'Submitters', 'Organization', 'Org_location',
       'Release_Date', 'Pangolin', 'PangoVersions', 'Surveillance_Sampling',
       'Isolate', 'Species', 'Genus', 'Family', 'Molecule_type', 'Length',
       'Nuc_Completeness', 'Genotype', 'Segment', 'Publications',
       'Geo_Location', 'Country', 'USA', 'Host', 'Tissue_Specimen_Source',
       'Collection_Date', 'BioSample', 'BioProject', 'GenBank_Title'],
      dtype='object')
  accession_id                                    Organism_Name  \
0   PV126503.1  Severe acute respiratory syndrome coronavirus 2   
1   PV126504.1  Severe acute respiratory syndrome coronavirus 2   
2   PV126506.1  Severe acute respiratory syndrome coronavirus 2   
3   PV126509.1  Severe acute respiratory syndrome coronavirus 2   
4   PV126510.1  Severe acute respiratory syndrome coronavirus 2   

  GenBank_RefSeq Assembly SRA_Accession  \
0        GenBank      NaN   

In [30]:
# List all .acc files in the S3 bucket
acc_files = [obj['Key'] for obj in response.get('Contents', []) if obj['Key'].endswith('.acc')]

# Read the .acc file
acc_file_key = acc_files[0]  # Pick the first .acc file in the list
acc_obj = s3_client.get_object(Bucket=s3_bucket, Key=acc_file_key)
acc_content = acc_obj["Body"].read().decode("utf-8")

In [31]:
# Split into a list of accession IDs
acc_list = acc_content.splitlines()

# Convert acc_list to DataFrame
acc_df = pd.DataFrame(acc_list, columns=['accession_id'])

# Inspect the accession DataFrame
print(acc_df.columns)
print(acc_df.head())

Index(['accession_id'], dtype='object')
  accession_id
0   PV126503.1
1   PV126504.1
2   PV126506.1
3   PV126509.1
4   PV126510.1


In [32]:
# List all FASTA files in the S3 bucket
fasta_files = [obj['Key'] for obj in response.get('Contents', []) if obj['Key'].endswith('.fasta')]

# Dictionary to store FASTA sequences
# Read and process each FASTA file
fasta_data = {}

In [ ]:
for fasta_file_key in fasta_files:
    fasta_obj = s3_client.get_object(Bucket=s3_bucket, Key=fasta_file_key)
    fasta_content = fasta_obj["Body"].read().decode("utf-8")
    
    # Extract Accession ID from the first line of the FASTA file
    accession_id = fasta_content.split("\n")[0].split()[0].replace(">", "")
    
    # Store the sequence by accession ID
    fasta_data[accession_id] = fasta_content

In [ ]:
# Convert fasta_data dictionary to a DataFrame
fasta_df = pd.DataFrame(list(fasta_data.items()), columns=['accession_id', 'fasta_sequence'])
# Inspect the FASTA DataFrame
print(fasta_df.columns)
print(fasta_df.head())

In [ ]:
# Merge all datasets on 'accession_id'
merged_df = acc_df.merge(metadata_df, on='accession_id', how='left').merge(fasta_df, on='accession_id', how='left')

# Inspect the merged DataFrame
print(merged_df.columns)

In [ ]:
# Verify if the data was merged successfully
# Display first few rows
print(merged_df.head())

# Check the shape (rows, columns)
print("\nShape of merged dataset:", merged_df.shape)

In [ ]:
# Check the number of sequences in the dataset
print("\nNumber of unique sequences:", merged_df['fasta_sequence'].nunique())

In [ ]:
# Check the number of unique accession ids in the merged dataset
print("\nUnique accession IDs:", merged_df['accession_id'].nunique())

In [ ]:
# Check for missingness in all columns
print("\nMissing values per column:\n", merged_df.isnull().sum())

In [ ]:
# Check if there are duplicate accession ids
print("\nDuplicate accession IDs:", merged_df['accession_id'].duplicated().sum())

In [ ]:
# Check datatypes of all columns in the merged dataset
print("\nColumn data types:\n", merged_df.dtypes)

In [ ]:
# Convert date columns to datetime formats
merged_df['Collection_Date'] = pd.to_datetime(merged_df['Collection_Date'], errors='coerce')

In [ ]:
# Check sequence length
merged_df['sequence_length'] = merged_df['fasta_sequence'].str.len()
print("\nSummary of sequence lengths:\n", merged_df['sequence_length'].describe())

In [ ]:
# Identify and remove any outliers in sequence length based on Q1, Q3 and Inter Quartile Range
Q1 = merged_df['sequence_length'].quantile(0.25)
Q3 = merged_df['sequence_length'].quantile(0.75)
IQR = Q3 - Q1
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

outliers = merged_df[(merged_df['sequence_length'] < lower_bound) | (merged_df['sequence_length'] > upper_bound)]
print("\nNumber of outlier sequences:", outliers.shape[0])

# Remove outliers if necessary
# merged_df = merged_df[(merged_df['sequence_length'] >= lower_bound) & (merged_df['sequence_length'] <= upper_bound)]

In [ ]:
# Check unique organisms -- All of these should be SARS-CoV-2
# Cause only these with their relevant ID were downloaded from the NCBI website
print("\nUnique organism names:", merged_df['Organism_Name'].unique())

In [ ]:
# Check for genome completeness -- All of these should be complete
print("\nGenome completeness values:", merged_df['Nuc_Completeness'].value_counts())

In [ ]:
# Check for missing collection dates -- Ideally none of these should be missing
missing_dates = merged_df['Collection_Date'].isnull().sum()
print("\nMissing collection dates:", missing_dates)

In [ ]:
# Extract year from collection date
merged_df['Year'] = merged_df['Collection_Date'].dt.year
print("\nYear distribution:\n", merged_df['Year'].value_counts())

In [ ]:
# Check for what is the distribution of countries
print("\nTop 10 countries in dataset:\n", merged_df['Country'].value_counts().head(10))

In [ ]:
# Country names -- remove any leading or trailing spaces
# Capitalize the first letter of each word using str.title().
merged_df['Country'] = merged_df['Country'].str.strip().str.title()

In [ ]:
# Check pango lineage distribution
print("\nTop 10 Pango lineages:\n", merged_df['Pangolin'].value_counts().head(10))

In [ ]:
# Remove unnecessary columns
columns_to_drop = ['GenBank_RefSeq', 'Submitters', 'BioSample', 'BioProject', 'Publications']
merged_df.drop(columns=columns_to_drop, inplace=True)

In [ ]:
# Check for missing pangolin Lineages
missing_pango = merged_df['Pangolin'].isnull().sum()
print("\nMissing Pango lineage values:", missing_pango)

In [ ]:
# Encode categorical variables
merged_df['Country'] = merged_df['Country'].astype('category').cat.codes
merged_df['Pangolin'] = merged_df['Pangolin'].astype('category').cat.codes

In [ ]:
# Check feature correlations
# Select only numeric columns from the data
numeric_df = merged_df.select_dtypes(include=['number'])

# Calculate the correlation matrix for numeric columns only
corr_matrix = numeric_df.corr()

# Plot the heatmap
plt.figure(figsize=(10, 6))
sns.heatmap(corr_matrix, annot=True, cmap="coolwarm", fmt=".2f")
plt.show()

In [ ]:
# Check feature distributions
merged_df.hist(figsize=(12, 8), bins=30)
plt.show()

In [ ]:
# Convert to lower case for consistency
merged_df.columns = merged_df.columns.str.lower()

In [ ]:
# Check memory usage if need be
# print("\nMemory usage before optimization:")
# print(merged_df.memory_usage(deep=True))

In [ ]:
# Save cleaned data if need be
# merged_df.to_csv("cleaned_dataset.csv", index=False)

# Download the CSV file to PC if required
# files.download('cleaned_dataset.csv')

In [ ]:
# Check column names in the DataFrame
print(merged_df.columns)

In [ ]:
# Collection_date should be in the datetime format
merged_df['collection_date'] = pd.to_datetime(merged_df['collection_date'], errors='coerce')

# Extract the Year from the collection_date
merged_df['Year'] = merged_df['collection_date'].dt.year

# Plot the boxplot
plt.figure(figsize=(10, 5))
sns.boxplot(x=merged_df['Year'], y=merged_df['sequence_length'])
plt.xticks(rotation=45)
plt.title("Genome Sequence Length Distribution by Year")
plt.xlabel("Year")
plt.ylabel("Sequence Length")
plt.show()

In [ ]:
# Distribution of sequence collected by month
merged_df['Month'] = merged_df['collection_date'].dt.month

plt.figure(figsize=(10, 5))
sns.countplot(x=merged_df['Month'], palette='coolwarm')
plt.xticks(range(0, 12), ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec'])
plt.title("Distribution of Collected Sequences by Month")
plt.xlabel("Month")
plt.ylabel("Number of Sequences")
plt.show()

In [ ]:
# Most frequently found SARS-CoV-2 lineages
# Top 15 most common lineages
top_pango = merged_df['pangolin'].value_counts().head(15)

plt.figure(figsize=(12, 5))
sns.barplot(x=top_pango.index, y=top_pango.values, palette="magma")
plt.xticks(rotation=45)
plt.title("Top 15 SARS-CoV-2 Lineages in Dataset")
plt.xlabel("Pango Lineage")
plt.ylabel("Count")
plt.show()

In [ ]:
# Most frequently found host species
top_hosts = merged_df['host'].value_counts().head(10)

plt.figure(figsize=(10, 5))
sns.barplot(x=top_hosts.index, y=top_hosts.values, palette="viridis")
plt.xticks(rotation=45)
plt.title("Most Common Host Species for SARS-CoV-2")
plt.xlabel("Host Species")
plt.ylabel("Count")
plt.show()

In [ ]:
# Geographic distribution of sequences
# Map
# First we will need to load the world map data from Natural Earth
world = gpd.read_file("https://naturalearth.s3.amazonaws.com/110m_cultural/ne_110m_admin_0_countries.zip")

# Count sequences per country
country_counts = merged_df['country'].value_counts().reset_index()
country_counts.columns = ['country', 'Count']

In [ ]:
# Make sure both columns are strings before merging
world['NAME'] = world['NAME'].astype(str)
country_counts['country'] = country_counts['country'].astype(str)

# Merge world map with sequence data
world = world.merge(country_counts, left_on="NAME", right_on="country", how="left")

# Plot
fig, ax = plt.subplots(figsize=(12, 6))
world.boundary.plot(ax=ax, linewidth=1, color="black")  # Plot country boundaries
world.plot(column='Count', cmap='OrRd', linewidth=0.5, edgecolor='black', legend=True, ax=ax)

plt.title("Geographic Distribution of SARS-CoV-2 Sequences")
plt.show()

In [ ]:
# Relationship between sequence length and pango lineage
plt.figure(figsize=(12, 6))
sns.boxplot(x=merged_df['pangolin'], y=merged_df['sequence_length'])
plt.xticks(rotation=90)
plt.title("Genome Lengths Across Pango Lineages")
plt.xlabel("Pango Lineage")
plt.ylabel("Sequence Length")
plt.show()

In [ ]:
# Lineage evolution over time
lineage_over_time = merged_df.groupby(['year', 'pangolin']).size().reset_index(name='count')

plt.figure(figsize=(12, 6))
sns.lineplot(data=lineage_over_time, x='year', y='count', hue='pangolin', marker='o', legend=None)
plt.title("SARS-CoV-2 Lineages Over Time")
plt.xlabel("Year")
plt.ylabel("Number of Sequences")
plt.show()

In [ ]:
# Sequence completeness vs. length
plt.figure(figsize=(8, 5))
sns.boxplot(x=merged_df['nuc_completeness'], y=merged_df['sequence_length'])
plt.title("Genome Lengths by Completeness")
plt.xlabel("Nucleotide Completeness")
plt.ylabel("Sequence Length")
plt.show()

In [ ]:
# Frequency of sampling by country over time
plt.figure(figsize=(12, 6))
sns.histplot(data=merged_df, x='year', hue='country', multiple="stack", palette="tab10")
plt.title("Sequence Contributions by Country Over Time")
plt.xlabel("Year")
plt.ylabel("Number of Sequences")
plt.show()

In [ ]:
# Sequence length distribution by country
plt.figure(figsize=(12, 6))
sns.boxplot(x=merged_df['country'], y=merged_df['sequence_length'])
plt.xticks(rotation=90)
plt.title("Genome Lengths Across Countries")
plt.xlabel("Country")
plt.ylabel("Sequence Length")
plt.show()